# Exodus v2025: Modular Crypto Data Pipeline

## Project Overview

Exodus v2025 is a modular data pipeline system for financial data, focused on cryptocurrencies but easily adaptable to other asset classes. The project automates the acquisition, cleaning, validation, splitting, and storage of market data, ensuring high data quality, traceability, and reproducibility.

## Key Features
- **Data Acquisition:** Download historical price and volume data from providers like Bybit and Binance.
- **Data Processing:** Clean, normalize, and enrich data with technical indicators.
- **Validation:** Ensure data integrity and quality through automated validation steps.
- **Data Splitting:** Prepare datasets for machine learning with flexible splitting strategies (chronological, random, sliding windows).
- **Storage:** Store processed data in TimescaleDB/PostgreSQL and export to Parquet, CSV, or JSON files.
- **Metadata Management:** Track dataset lineage and quality with structured metadata.
- **Configuration Management:** Easily configure and orchestrate pipelines for different assets, timeframes, and storage options.
- **Structured Logging:** All processes are logged for diagnostics, auditing, and monitoring.
- **Automated Testing:** Includes unit and integration tests to ensure reliability.

## Technologies Used
- Python (Pandas, logging, pytest)
- PostgreSQL / TimescaleDB
- Modular architecture (providers, processing, validation, storage)

## Example Workflow
1. **Configure the pipeline:** Define which provider, symbol, timeframe, and validation rules to use.
2. **Run the pipeline:** Automatically download, clean, validate, split, and store the data.
3. **Review results:** Check logs, validation reports, and stored datasets for quality and completeness.

## Why is this project valuable?
- Demonstrates end-to-end data engineering skills: ETL, validation, storage, and automation.
- Shows best practices in modular design, testing, and logging.
- Ready to be extended for new data sources or analytics needs.

## Repository
[GitHub - Exodus v2025](https://github.com/your-username/exodus-v2025)

---
*Developed by Jose Acosta Available for junior data engineering and analytics roles.* 

In [14]:
from pipelines.crypto_pipeline import CryptoPipeline
from pipelines.config_manager import PipelineConfig

ImportError: cannot import name 'BybitDownloader' from 'src.data.providers.crypto.bybit_downloader' (c:\Users\DataBridge\Documents\workspace\Exodus_V2025\src\data\providers\crypto\bybit_downloader.py)

In [ ]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
